In [1]:
import os

import numpy as np
from keras import Model
from tqdm import tqdm

from src.dlla.hw import dlla_hw, prepare_traces_dl
from src.dlla.model9 import make_model_9
from src.dlla.wegener import make_mlp_wegener, binomial_test
from src.pollution.clock_jitter import clock_jitter
from src.pollution.gaussian_noise import gaussian_noise
from src.pollution.random_delay import random_delay
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW
from src.trace_set.transform import reduce_fixed_fixed, fixed_fixed
from src.tvla.cri import tvla_cri

In [2]:
DB = Database.aisy

In [3]:
RAW_TRACES = TraceSetHW(DB)

X_PROF_CXT, Y_PROF = RAW_TRACES.profile_states()
X_ATT_CXT, Y_ATT = RAW_TRACES.attack_states()

A_THIRD = round(X_PROF_CXT.shape[1] / 3)

START = A_THIRD
END = A_THIRD * 2
SAMPLE_TRACE = X_PROF_CXT[0][START:END]

X_PROF_CXT.shape

(6667, 7332)

In [4]:
X_PROF = X_PROF_CXT[:, START:END]
X_ATT = X_ATT_CXT[:, START:END]

X_PROF.shape

(6667, 2444)

In [ ]:
WINDOW_JITTER_PARAMS = np.arange(0, 100, 50)

def desync(traces: np.ndarray, window: (int, int), sigma: float):
    start, end = window
    num_traces = len(traces)
    num_sample_points = end - start

    permutations = np.round(np.random.normal(scale=sigma, size=num_traces)).astype(int)
    permutations += start

    res = np.ones((num_traces, num_sample_points), dtype=traces.dtype)

    for ix in tqdm(range(num_traces), f"Trace desynchronization, sigma={sigma}"):
        permutation = permutations[ix]
        res[ix] = traces[ix, permutation:permutation + num_sample_points]

    return res

noise_functions = {
    PollutionType.gauss: gaussian_noise,
    PollutionType.desync: lambda traces, p: desync(traces, WINDOW, p),
}

GAUSS_PARAMS = np.arange(0, 82, 2)

def apply_pollution(poll_type: PollutionType, params: list, db: Database):
    default = TraceSetHW(db)
    x, state = default.profile_states()
    x_att, state_att = default.attack_states()

    for param in params:
        pollution = Pollution(poll_type, param)
        out = TraceSetHW(db, pollution)

        if not os.path.exists(out.path):
            f = noise_functions[poll_type]
            noise_x = f(x, param)
            noise_x_att = f(x_att, param)

            out.create(noise_x, state, noise_x_att, state_att)

            x9, y9, x9_att, y9_att = prepare_traces_dl(*out.profile(), *out.attack())
            mdl9 = make_model_9(x9, y9, x9_att, y9_att)
            p_value = dlla_hw(mdl9, x9_att, y9_att)

            print(f"{pollution.get_name()} ({param}): p-value: ({p_value}).")

apply_pollution(PollutionType.delay, np.arange(0.00015, 0.00025, 0.000005), Database.ascad)

Random delay (0.000155):  10%|▉         | 19600/200000 [00:10<01:38, 1824.93it/s]

In [7]:
str(0.00015).replace('.','-')

str(float(f"{0.00015999999999999999:.8f}"))

'0.00016'

### TVLA vs. DL-LA

In [6]:
ORDER = 2

def store_results(database: Database, method: str, pollution: Pollution, p):
    file_name = f"results_{database.name}.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{pollution.type.name};{pollution.parameter};{p}\n")

In [7]:
def wegener_p(mdl: Model, x_att: np.ndarray, y_att: np.ndarray):
    predictions = mdl.predict(x_att).argmax(axis=1)
    labels = y_att.argmax(axis=1)

    correct = np.sum(predictions == labels)
    total = len(predictions)
    # print(total, correct)

    return binomial_test(total, correct)

LIMIT_PROF, LIMIT_ATT = None, 100000

def la_benchmark(db: Database, pollution_type: PollutionType, params):
    for param in params:
        pollution = Pollution(pollution_type, param)
        print("Load traces         ", end="\r")
        trace_set = TraceSetHW(db, pollution, (LIMIT_PROF, LIMIT_ATT))

        if os.path.exists(trace_set.path):
            print("Prepare traces (1/2)", end="\r")
            x9, y9, x9_att, y9_att = prepare_traces_dl(*trace_set.profile(), *trace_set.attack())
            print("Prepare traces (2/2)", end="\r")
            (x2, y2), (x2_att, y2_att) = reduce_fixed_fixed(x9, y9), reduce_fixed_fixed(x9_att, y9_att)

            print("Make model (1/2)    ", end="\r")
            mdl9 = make_mlp(x9, y9)
            print("Make model (2/2)    ", end="\r")
            mdl2 = make_mlp_wegener(x2, y2, False)

            print("TVLA                ", end="\r")
            tvla_ps = np.min(tvla_cri(*fixed_fixed(*trace_set.profile()), ORDER), axis=1)

            print("Predict             ", end="\r")
            dlla9_p = dlla_hw(mdl9, x9_att, y9_att)
            dlla2_p = wegener_p(mdl2, x2_att, y2_att)

            print(f"{pollution_type} ({param}). TVLA ({tvla_ps}). DLLA9 ({dlla9_p}). DLLA2 ({dlla2_p})", end="\r")

            for order, p in enumerate(tvla_ps):
                if order > 0:
                    store_results(db, f"cri_tvla_{order}", pollution, p)

            store_results(db, "dlla9", pollution, dlla9_p)
            store_results(db, "dlla2", pollution, dlla2_p)

        print()

while True:
    la_benchmark(DB, PollutionType.gauss, GAUSS_PARAMS)


NameError: name 'make_mlp' is not defined